In [157]:
import pandas as pd #Importing all the needed libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [158]:
data=pd.read_csv('training_variants') #Importing the data
text_data=pd.read_csv('training_text',sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)

In [159]:
data.shape #data has 3321 data points

(3321, 4)

In [160]:
data.columns  #has following features where class is dependent variable

Index(['ID', 'Gene', 'Variation', 'Class'], dtype='object')

In [161]:
text_data.shape #Text ahs 2 features one is id and other is actual text.

(3321, 2)

In [162]:
data=data.join(text_data,lsuffix='_') #Joining the data.

In [163]:
data[data['Variation']=='Truncating Mutations']['Class'].value_counts() #Just a insight that specific type of variation belongs to which group.

1    89
6     2
4     1
2     1
Name: Class, dtype: int64

In [164]:
data.Class.value_counts() #Total classes 1-9.(9 classes)

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
Name: Class, dtype: int64

In [165]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
enc1=OneHotEncoder() #importing the encoders

In [167]:
encoded=enc.fit_transform(data['Gene'].values.reshape(-1,1))
add=data.shape[1]
for i in range(0,encoded.shape[1]):
    data[add] = pd.arrays.SparseArray(encoded[:, i].toarray().ravel())
    add+=1 #Converting the Gene feature as one hot encoded

In [168]:
encoded=enc1.fit_transform(data['Variation'].values.reshape(-1,1))
add=data.shape[1]
for i in range(0,encoded.shape[1]):
    data[add] = pd.arrays.SparseArray(encoded[:, i].toarray().ravel())
    add+=1  #Converting the Gene feature as one hot encoded

In [171]:
def preprocess(x): #Basic pre-processing the text feature
        x=str(x).lower()
        x=x.replace(',000,000','m').replace(',000','k').replace("'","`").replace("won't",'will not').replace("can't","can not").replace('cannot','can not').replace("n't",'not').replace("what's","what is").replace("it's","it is").replace("'ve","have").replace("i'm","i am").replace("'re","are").replace("he's","he is").replace("she's","she is ").replace("'s","own").replace("%"," precent").replace("₹"," rupeee").replace("$","dollar").replace("€","euro").replace("'ll","will")
        porter = PorterStemmer()
        pattern = re.compile('\W')
    
        if type(x) == type(''):
            x = re.sub(pattern, ' ', x)
        if type(x) == type(''):
          x = porter.stem(x)
          example1 = BeautifulSoup(x)
          x = example1.get_text()
        return str(x)

In [172]:
data['text']=data['TEXT'].apply(preprocess) #applying the text pre-processing to each feature.

In [174]:
data.drop(columns=['Gene','Variation','ID','ID_'],inplace=True) #Dropping the already encoded features.

In [176]:
X=data.iloc[:,1:] #getting X and y seperate
y=data.iloc[:,0]

In [178]:
from sklearn.model_selection import train_test_split #Splitting the data.
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [179]:
X_train.reset_index(drop=True,inplace=True) #reset_index for looping conventions.

In [181]:
import gensim
# Load Google's pre-trained Word2Vec model.
w2v_model_our_corpus = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\Roggh\Desktop\AAIC\Twitter_sentiment\GoogleNews-vectors-negative300.bin', binary=True)  

In [182]:
def sent_vectorizer(sent, w2v_model_our_corpus): #Get the sentence vectors
    sent_vec = np.zeros(300)
    numw = 0
    for w in sent.split():
        try:
            sent_vec = np.add(sent_vec, w2v_model_our_corpus[w])
            numw+=1
        except:
            pass
    return sent_vec/ np.sqrt(sent_vec.dot(sent_vec))

In [183]:
from tqdm import tqdm #Converting the sentences into vectors. 
V=[]
for sentence in tqdm(X_train['text']):
    V.append(sent_vectorizer(sentence, w2v_model_our_corpus))
V_test=[]
for sentence in tqdm(X_test['text']):
    V_test.append(sent_vectorizer(sentence, w2v_model_our_corpus))
#csr_V_test=tf.convert_to_tensor(V_test)
V_test=np.array(V_test)

100%|████████████████████████████████████████████████████████████████████████████████| 831/831 [00:56<00:00, 14.41it/s]


In [184]:
X_train.drop(columns=['TEXT','text'],axis=1,inplace=True) #dropping not needed features from train set

C:\Users\Roggh\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [185]:
X_test.drop(columns=['TEXT','text'],axis=1,inplace=True) #dropping not needed features from test set

In [186]:
from scipy.sparse import hstack #Stacking one-hot encoded Gene and variation feature with the sentence vectors whihc we got from 
X_train=hstack([X_train,V]) #Pre-trained Word2Vec model. 
X_test=hstack([X_test,V_test])

In [187]:
from catboost import CatBoostClassifier  #Importing the classifier.
lr=CatBoostClassifier()

In [201]:
lr.fit(X_train,y_train) #Fitting the function-1000 epochs(iterations).

Learning rate set to 0.082895
0:	learn: 2.1024383	total: 1.15s	remaining: 19m 8s
1:	learn: 2.0164730	total: 1.82s	remaining: 15m 9s
2:	learn: 1.9415529	total: 2.53s	remaining: 14m 1s
3:	learn: 1.8826003	total: 3.25s	remaining: 13m 28s
4:	learn: 1.8253776	total: 3.94s	remaining: 13m 3s
5:	learn: 1.7817428	total: 4.63s	remaining: 12m 46s
6:	learn: 1.7367444	total: 5.38s	remaining: 12m 43s
7:	learn: 1.6997227	total: 6.06s	remaining: 12m 31s
8:	learn: 1.6623813	total: 6.79s	remaining: 12m 27s
9:	learn: 1.6256194	total: 7.53s	remaining: 12m 25s
10:	learn: 1.5931340	total: 8.23s	remaining: 12m 20s
11:	learn: 1.5624602	total: 8.97s	remaining: 12m 18s
12:	learn: 1.5340751	total: 9.66s	remaining: 12m 13s
13:	learn: 1.5080429	total: 10.3s	remaining: 12m 7s
14:	learn: 1.4866286	total: 11s	remaining: 12m 4s
15:	learn: 1.4645224	total: 11.7s	remaining: 11m 59s
16:	learn: 1.4387722	total: 12.4s	remaining: 11m 54s
17:	learn: 1.4213014	total: 13.1s	remaining: 11m 52s
18:	learn: 1.4031134	total: 13.7s	

154:	learn: 0.7945156	total: 1m 49s	remaining: 9m 58s
155:	learn: 0.7935557	total: 1m 50s	remaining: 9m 57s
156:	learn: 0.7926802	total: 1m 51s	remaining: 9m 56s
157:	learn: 0.7925431	total: 1m 51s	remaining: 9m 55s
158:	learn: 0.7923357	total: 1m 52s	remaining: 9m 54s
159:	learn: 0.7922672	total: 1m 53s	remaining: 9m 53s
160:	learn: 0.7918805	total: 1m 53s	remaining: 9m 52s
161:	learn: 0.7917224	total: 1m 54s	remaining: 9m 51s
162:	learn: 0.7912840	total: 1m 54s	remaining: 9m 50s
163:	learn: 0.7903153	total: 1m 55s	remaining: 9m 49s
164:	learn: 0.7886375	total: 1m 56s	remaining: 9m 48s
165:	learn: 0.7885273	total: 1m 57s	remaining: 9m 48s
166:	learn: 0.7883222	total: 1m 57s	remaining: 9m 47s
167:	learn: 0.7852634	total: 1m 58s	remaining: 9m 47s
168:	learn: 0.7851844	total: 1m 59s	remaining: 9m 46s
169:	learn: 0.7834055	total: 2m	remaining: 9m 46s
170:	learn: 0.7817880	total: 2m 1s	remaining: 9m 47s
171:	learn: 0.7817138	total: 2m 2s	remaining: 9m 47s
172:	learn: 0.7816321	total: 2m 2s

307:	learn: 0.7104446	total: 3m 52s	remaining: 8m 41s
308:	learn: 0.7085193	total: 3m 52s	remaining: 8m 40s
309:	learn: 0.7084437	total: 3m 53s	remaining: 8m 39s
310:	learn: 0.7072479	total: 3m 54s	remaining: 8m 38s
311:	learn: 0.7070695	total: 3m 54s	remaining: 8m 38s
312:	learn: 0.7069994	total: 3m 55s	remaining: 8m 37s
313:	learn: 0.7050862	total: 3m 56s	remaining: 8m 36s
314:	learn: 0.7050215	total: 3m 57s	remaining: 8m 35s
315:	learn: 0.7049400	total: 3m 57s	remaining: 8m 34s
316:	learn: 0.7040629	total: 3m 58s	remaining: 8m 34s
317:	learn: 0.7039877	total: 3m 59s	remaining: 8m 33s
318:	learn: 0.7039265	total: 4m	remaining: 8m 32s
319:	learn: 0.7014090	total: 4m	remaining: 8m 31s
320:	learn: 0.7010578	total: 4m 1s	remaining: 8m 31s
321:	learn: 0.7009870	total: 4m 2s	remaining: 8m 30s
322:	learn: 0.7009283	total: 4m 3s	remaining: 8m 29s
323:	learn: 0.7008290	total: 4m 3s	remaining: 8m 28s
324:	learn: 0.7007548	total: 4m 4s	remaining: 8m 27s
325:	learn: 0.7006841	total: 4m 5s	remain

460:	learn: 0.6749523	total: 5m 39s	remaining: 6m 36s
461:	learn: 0.6748880	total: 5m 39s	remaining: 6m 35s
462:	learn: 0.6744329	total: 5m 40s	remaining: 6m 34s
463:	learn: 0.6743085	total: 5m 41s	remaining: 6m 34s
464:	learn: 0.6742368	total: 5m 41s	remaining: 6m 33s
465:	learn: 0.6741772	total: 5m 42s	remaining: 6m 32s
466:	learn: 0.6741067	total: 5m 43s	remaining: 6m 31s
467:	learn: 0.6739773	total: 5m 43s	remaining: 6m 30s
468:	learn: 0.6738777	total: 5m 44s	remaining: 6m 30s
469:	learn: 0.6738060	total: 5m 45s	remaining: 6m 29s
470:	learn: 0.6737570	total: 5m 45s	remaining: 6m 28s
471:	learn: 0.6736924	total: 5m 46s	remaining: 6m 27s
472:	learn: 0.6736290	total: 5m 47s	remaining: 6m 26s
473:	learn: 0.6735644	total: 5m 48s	remaining: 6m 26s
474:	learn: 0.6728055	total: 5m 48s	remaining: 6m 25s
475:	learn: 0.6727355	total: 5m 49s	remaining: 6m 24s
476:	learn: 0.6726680	total: 5m 50s	remaining: 6m 23s
477:	learn: 0.6726053	total: 5m 50s	remaining: 6m 23s
478:	learn: 0.6725313	total:

614:	learn: 0.6517112	total: 7m 26s	remaining: 4m 39s
615:	learn: 0.6516305	total: 7m 27s	remaining: 4m 38s
616:	learn: 0.6515656	total: 7m 28s	remaining: 4m 38s
617:	learn: 0.6515066	total: 7m 28s	remaining: 4m 37s
618:	learn: 0.6494801	total: 7m 29s	remaining: 4m 36s
619:	learn: 0.6494116	total: 7m 30s	remaining: 4m 35s
620:	learn: 0.6493583	total: 7m 30s	remaining: 4m 35s
621:	learn: 0.6492921	total: 7m 31s	remaining: 4m 34s
622:	learn: 0.6492256	total: 7m 32s	remaining: 4m 33s
623:	learn: 0.6491637	total: 7m 32s	remaining: 4m 32s
624:	learn: 0.6491020	total: 7m 33s	remaining: 4m 32s
625:	learn: 0.6490475	total: 7m 34s	remaining: 4m 31s
626:	learn: 0.6489865	total: 7m 35s	remaining: 4m 30s
627:	learn: 0.6489125	total: 7m 36s	remaining: 4m 30s
628:	learn: 0.6488459	total: 7m 36s	remaining: 4m 29s
629:	learn: 0.6487857	total: 7m 37s	remaining: 4m 28s
630:	learn: 0.6487287	total: 7m 38s	remaining: 4m 27s
631:	learn: 0.6486030	total: 7m 38s	remaining: 4m 27s
632:	learn: 0.6485339	total:

767:	learn: 0.6261504	total: 9m 2s	remaining: 2m 43s
768:	learn: 0.6244739	total: 9m 3s	remaining: 2m 43s
769:	learn: 0.6241819	total: 9m 3s	remaining: 2m 42s
770:	learn: 0.6241212	total: 9m 4s	remaining: 2m 41s
771:	learn: 0.6240585	total: 9m 4s	remaining: 2m 40s
772:	learn: 0.6228130	total: 9m 5s	remaining: 2m 40s
773:	learn: 0.6227520	total: 9m 6s	remaining: 2m 39s
774:	learn: 0.6227018	total: 9m 6s	remaining: 2m 38s
775:	learn: 0.6226476	total: 9m 7s	remaining: 2m 38s
776:	learn: 0.6225714	total: 9m 8s	remaining: 2m 37s
777:	learn: 0.6225172	total: 9m 8s	remaining: 2m 36s
778:	learn: 0.6224595	total: 9m 9s	remaining: 2m 35s
779:	learn: 0.6224067	total: 9m 9s	remaining: 2m 35s
780:	learn: 0.6207953	total: 9m 10s	remaining: 2m 34s
781:	learn: 0.6207299	total: 9m 11s	remaining: 2m 33s
782:	learn: 0.6206338	total: 9m 11s	remaining: 2m 32s
783:	learn: 0.6203703	total: 9m 12s	remaining: 2m 32s
784:	learn: 0.6203138	total: 9m 13s	remaining: 2m 31s
785:	learn: 0.6202634	total: 9m 13s	remai

920:	learn: 0.5987553	total: 10m 37s	remaining: 54.7s
921:	learn: 0.5987033	total: 10m 37s	remaining: 54s
922:	learn: 0.5986419	total: 10m 38s	remaining: 53.3s
923:	learn: 0.5985908	total: 10m 39s	remaining: 52.6s
924:	learn: 0.5985399	total: 10m 39s	remaining: 51.9s
925:	learn: 0.5972624	total: 10m 40s	remaining: 51.2s
926:	learn: 0.5972114	total: 10m 40s	remaining: 50.5s
927:	learn: 0.5971595	total: 10m 41s	remaining: 49.8s
928:	learn: 0.5970995	total: 10m 42s	remaining: 49.1s
929:	learn: 0.5969742	total: 10m 42s	remaining: 48.4s
930:	learn: 0.5969083	total: 10m 43s	remaining: 47.7s
931:	learn: 0.5968550	total: 10m 43s	remaining: 47s
932:	learn: 0.5943531	total: 10m 44s	remaining: 46.3s
933:	learn: 0.5943001	total: 10m 45s	remaining: 45.6s
934:	learn: 0.5934016	total: 10m 45s	remaining: 44.9s
935:	learn: 0.5933449	total: 10m 46s	remaining: 44.2s
936:	learn: 0.5932648	total: 10m 47s	remaining: 43.5s
937:	learn: 0.5932105	total: 10m 47s	remaining: 42.8s
938:	learn: 0.5931499	total: 10m

In [202]:
lr.score(X_test,y_test) #prediction accuracy is 65% 

0.6594464500601684

In [203]:
preds=lr.predict_proba(X_test)

In [204]:
from sklearn.metrics import log_loss
log_loss(y_test,preds) #but the log-loss is very less as our aim is to improve on prediction robability this model is good.

1.0182210246583014

In [205]:
from sklearn.metrics import log_loss
log_loss(y_train,lr.predict_proba(X_train)) #also not overfitting :)

0.5869114702957151